In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Distributed Training with Reduction Server

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This notebook demonstrates how to optimize large distributed training jobs with the Vertex Training Reduction Server. The [TensorFlow NLP Modelling Toolkit](https://github.com/tensorflow/models/tree/master/official/nlp#tensorflow-nlp-modelling-toolkit) from the [TensorFlow Model Garden](https://github.com/tensorflow/models) is used to preprocess the data and create the model and training loop. `MultiWorkerMirroredStrategy` from the `tf.distribute` module is used to distribute model training across multiple machines. 

For more information about using Reduction Server to optimize distributed training refer to the [Optimizing Distributed Training with Vertex AI Reduction Server](tbd) article.

### Dataset

The dataset used for this tutorial is the [Multi-Genre Natural Language Inference Corpus (MNLI)](https://www.tensorflow.org/datasets/catalog/glue#gluemnli) from the GLUE benchmark. This dataset is loaded from [TensorFlow Datasets](https://www.tensorflow.org/datasets), and used to fine tune a BERT model for sentence prediction.


### Objective

In this notebook, you create a custom-trained model from a Python script in a Docker container. You learn how to configure, submit, and monitor a Vertex Training job that uses Reduction Server to optimize network bandwith and latency of the gradient reduction operation in distributed training.  

The steps performed include:

- Convert the MNLI dataset to the format required by the TensorFlow NLP Modelling Toolkit.
- Create a Vertex AI custom job that uses Reduction Server.
- Submit and monitor the job.
- Cleanup resources.

### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install the required packages

Install the TensorFlow Official Models and TensorFlow Text libraries.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
! pip3 install --upgrade tf-models-official==2.5.0 tensorflow-text==2.5.0

Install the latest version of the Vertex SDK for Python.

In [3]:
! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest version of the Google Cloud Storage library.

In [4]:
! pip3 install --upgrade google-cloud-storage $USER_FLAG

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [5]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin


### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  curious-entropy-222019


Otherwise, set your project ID here.

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "curious-entropy-222019"  # @param {type:"string"}
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [3]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**


In this example, your training application uses Cloud Storage for accessing training and validation datasets and for storing checkpoints. 

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [4]:
BUCKET_NAME = "gs://jk-rs-testing"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [5]:
! gsutil mb -l $REGION $BUCKET_NAME

Creating gs://jk-rs-testing/...
ServiceException: 409 A Cloud Storage bucket named 'jk-rs-testing' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [6]:
! gsutil ls -al $BUCKET_NAME

                                 gs://jk-rs-testing/MNLI_20210710_210507/
                                 gs://jk-rs-testing/datasets/


### Import libraries and define constants

In [7]:
import json
import pprint
import time
import tensorflow as tf

from official.nlp.bert import tokenization
from official.nlp.data import classifier_data_lib

from google.cloud import aiplatform
from google.cloud.aiplatform_v1beta1 import types
from google.cloud.aiplatform_v1beta1.services.job_service import JobServiceClient

### Set up variables

Next, set up some variables used throughout the tutorial.

- API_ENDPOINT: The Vertex API service endpoint for job services.
- PARENT: The Vertex location root path for job resources.

In [8]:
API_ENDPOINT = '{}-aiplatform.googleapis.com'.format(REGION)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

### Set up clients

The Vertex client library works as a client/server model. In your Python script, you create a client that sends requests and receives responses from the Vertex server.

In this example, you use the Job Service client for submitting and monitoring custom training jobs.

In [9]:
client_options = {"api_endpoint": API_ENDPOINT}
job_client = JobServiceClient(client_options=client_options)

/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Prepare training and validation datasets

The TensorFlow NLP Modelling Toolkit provides reusable and modularized modeling building blocks. The following function uses utility functions from the toolkit to transform the MNLI dataset into the format expected by the BERT model.

In [10]:
def generate_mnli_tfrecords(
    train_data_output_path,
    eval_data_output_path,
    metadata_file_path,
    vocab_file='gs://cloud-tpu-checkpoints/bert/keras_bert/uncased_L-24_H-1024_A-16/vocab.txt',
    mnli_type='matched',
    max_seq_length=128,
    do_lower_case=True):
  """Generates MNLI training/validation splits in the TFRecord format compatible with the Modelling Toolkit."""

  tokenizer = tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

  processor_text_fn = tokenization.convert_to_unicode

  if mnli_type == 'matched':
    tfds_params = 'dataset=glue/mnli,text_key=hypothesis,text_b_key=premise,train_split=train,dev_split=validation_matched'
  else:
    tfds_params = 'dataset=glue/mnli,text_key=hypothesis,text_b_key=premise,train_split=train,dev_split=validation_mismatched'

  processor = classifier_data_lib.TfdsProcessor(
      tfds_params=tfds_params, process_text_fn=processor_text_fn)

  metadata = classifier_data_lib.generate_tf_record_from_data_file(
      processor,
      None,
      tokenizer,
      train_data_output_path=train_data_output_path,
      eval_data_output_path=eval_data_output_path,
      max_seq_length=max_seq_length)

  with tf.io.gfile.GFile(metadata_file_path, 'w') as writer:
    writer.write(json.dumps(metadata, indent=4) + '\n')

In [11]:
# Define data locations

OUTPUT_LOCATION = f'{BUCKET_NAME}/datasets/MNLI'
TRAIN_FILE = f'{OUTPUT_LOCATION}/mnli_train.tf_record'
EVAL_FILE = f'{OUTPUT_LOCATION}/mnli_valid.tf_record'
METADATA_FILE = f'{OUTPUT_LOCATION}/metadata.json'

In [12]:
generate_mnli_tfrecords(TRAIN_FILE, EVAL_FILE, METADATA_FILE)

INFO:absl:Load dataset info from /home/jupyter/tensorflow_datasets/glue/mnli/1.0.0
INFO:absl:Reusing dataset glue (/home/jupyter/tensorflow_datasets/glue/mnli/1.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split None, from /home/jupyter/tensorflow_datasets/glue/mnli/1.0.0
INFO:absl:Writing example 0 of 392702
INFO:absl:*** Example ***
INFO:absl:guid: train-0
INFO:absl:tokens: [CLS] meaningful partnerships with stakeholders is crucial . [SEP] in recognition of these tensions , l ##sc has worked dil ##igen ##tly since 1995 to convey the expectations of the state planning initiative and to establish meaningful partnerships with stakeholders aimed at foster ##ing a new sy ##mb ##ios ##is between the federal provider and recipients of legal services funding . [SEP]
INFO:absl:input_ids: 101 15902 13797 2007 22859 2003 10232 1012 102 1999 5038 1997 2122 13136 1010 1048 11020 2038 2499 29454 29206 14626 2144 2786 2000 16636 1996 10908 1997 1996 2110 4041 6349 1998 2000 5323 15902 13797

In [13]:
# Verify that the files were successfully created.

! gsutil ls {OUTPUT_LOCATION}

gs://jk-rs-testing/datasets/MNLI/
gs://jk-rs-testing/datasets/MNLI/metadata.json
gs://jk-rs-testing/datasets/MNLI/mnli_train.tf_record
gs://jk-rs-testing/datasets/MNLI/mnli_valid.tf_record


In [14]:
# Examine the metadata

! gsutil cat {METADATA_FILE}

{
    "processor_type": "TFDS_glue/mnli",
    "train_data_size": 392702,
    "max_seq_length": 128,
    "task_type": "bert_classification",
    "num_labels": 3,
    "eval_data_size": 9815
}


### Create a training container
#### Write Dockerfile

The first step in containerizing your code is to create a Dockerfile. In the Dockerfile, you'll include all the commands needed to run the image such as installing the necessary libraries and setting up the entry point for the training code.

This Dockerfile uses the Deep Learning Container TensorFlow Enterprise 2.5 GPU Docker image. The Deep Learning Containers on Google Cloud come with many common ML and data science frameworks pre-installed. After downloading that image, this Dockerfile installs the TensorFlow Official Models and TensorFlow Text libraries, and the Reduction Server NCCL plugin.

In [15]:
! rm -rf training_image
! mkdir training_image
! mkdir training_image/trainer

In [16]:
%%writefile training_image/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-5
WORKDIR /

# Install Reduction Server NCCL plugin
RUN apt remove -y google-fast-socket \
&&  echo "deb https://packages.cloud.google.com/apt google-fast-socket main" | tee /etc/apt/sources.list.d/google-fast-socket.list \
&&  curl -s -L https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - \
&&  apt update && apt install -y google-reduction-server

# Install Official Models and Text libraries
RUN pip install tf-models-official==2.5.0 tensorflow-text==2.5.0


# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python"]
CMD ["-c", "print('TF Model Garden')"]

Writing training_image/Dockerfile


#### Create training application code

Next, you create a `training_image/trainer` directory with a `train.py` script that contains the code for your training application.

The `train.py` training script is based on [the common training driver](https://github.com/tensorflow/models/blob/master/official/nlp/docs/train.md) from the TensorFlow NLP Modelling Toolkit. The common training driver script supports multiple NLP tasks (e.g., pre-training, GLUE and SQuAD fine-tuning) and multiple models (e.g., BERT, ALBERT). 

A set of configurations to use for a specific NLP task is called an experiment.  The [TensorFlow NLP Modelling Toolkit](https://github.com/tensorflow/models/tree/master/official/nlp#tensorflow-nlp-modelling-toolkit) includes a set of pre-defined experiments. When you invoke the script you specificy an experiment type using the `--experiment` command line parameter. The default experiment configuration can be overridden using two methods:
- You can specify one or multiple YAML configuration with updated settings using the `--config_file` command line parameter
- You can provide updated settings as a list of key/value pairs through the `--params_override` command line parameter

When you specify both the `--config_file` and the `--params_override`, the settings in the `--params_override` take precedence.

Retrieving the default experiment configuration and merging user provided settings is encapsulated in the `official.core.train_utils.parse_configuration` utility function from the [TensorFlow Model Garden](https://github.com/tensorflow/models).

The [common training driver](https://github.com/tensorflow/models/blob/master/official/nlp/train.py)  has been adapted to work seamlessly on a distributed compute environment provisioned when running a Vertex Training job. The [TensorFlow NLP Modelling Toolkit](https://github.com/tensorflow/models/tree/master/official/nlp#tensorflow-nlp-modelling-toolkit) uses [Orbit](https://github.com/tensorflow/models/tree/master/orbit) to implement a custom training loop. The custom training loop saves checkpoints, write Tensorboard summaries, and saves a trained model to a storage location specified through the `--model-dir` command line parameter. When using the base common training driver in a distributed setting each worker uses the same base storage location. To avoid conflicts when multiple workers write to the same storage location, we modified the code so that each worker uses a different storage location based on its role in a Vertex compute cluster. This logic is implemented by the `_get_model_dir` utility function.


In [17]:
%%writefile training_image/trainer/train.py

# Copyright 2021 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""TFM common training driver."""

import json
import os

from absl import app
from absl import flags
from absl import logging
import gin
from official.common import distribute_utils
from official.common import flags as tfm_flags
from official.common import registry_imports
from official.core import task_factory
from official.core import train_lib
from official.core import train_utils
from official.modeling import performance

FLAGS = flags.FLAGS


def _get_model_dir(model_dir):
  """Defines utility functions for model saving.

  In a multi-worker scenario, the chief worker will save to the
  desired model directory, while the other workers will save the model to
  temporary directories. It’s important that these temporary directories
  are unique in order to prevent multiple workers from writing to the same
  location. Saving can contain collective ops, so all workers must save and
  not just the chief.
  """

  def _is_chief(task_type, task_id):
    return ((task_type == 'chief' and task_id == 0) or task_type is None)

  tf_config = os.getenv('TF_CONFIG')
  if tf_config:
    tf_config = json.loads(tf_config)

    if not _is_chief(tf_config['task']['type'], tf_config['task']['index']):
      model_dir = os.path.join(model_dir,
                               'worker-{}').format(tf_config['task']['index'])

  logging.info('Setting model_dir to: %s', model_dir)

  return model_dir


def main(_):

  model_dir = _get_model_dir(FLAGS.model_dir)

  gin.parse_config_files_and_bindings(FLAGS.gin_file, FLAGS.gin_params)
  params = train_utils.parse_configuration(FLAGS)

  if 'train' in FLAGS.mode:
    # Pure eval modes do not output yaml files. Otherwise continuous eval job
    # may race against the train job for writing the same file.
    train_utils.serialize_config(params, model_dir)

  # Sets mixed_precision policy. Using 'mixed_float16' or 'mixed_bfloat16'
  # can have significant impact on model speeds by utilizing float16 in case of
  # GPUs, and bfloat16 in the case of TPUs. loss_scale takes effect only when
  # dtype is float16
  if params.runtime.mixed_precision_dtype:
    performance.set_mixed_precision_policy(params.runtime.mixed_precision_dtype)
  distribution_strategy = distribute_utils.get_distribution_strategy(
      distribution_strategy=params.runtime.distribution_strategy,
      all_reduce_alg=params.runtime.all_reduce_alg,
      num_gpus=params.runtime.num_gpus,
      tpu_address=params.runtime.tpu,
      **params.runtime.model_parallelism())
  with distribution_strategy.scope():
    task = task_factory.get_task(params.task, logging_dir=model_dir)

  train_lib.run_experiment(
      distribution_strategy=distribution_strategy,
      task=task,
      mode=FLAGS.mode,
      params=params,
      model_dir=model_dir)

  train_utils.save_gin_config(FLAGS.mode, model_dir)


if __name__ == '__main__':
  tfm_flags.define_flags()
  app.run(main)

Writing training_image/trainer/train.py


####  Create base settings for the GLUE/MNLI fine tuning experiment

The [TensorFlow NLP Modelling Toolkit](https://github.com/tensorflow/models/tree/master/official/nlp#tensorflow-nlp-modelling-toolkit) includes a predefined experiment for a set of text classification tasks - the `bert/sentence_prediction` experiment. The base settings in the `bert/sentence_prediction` experiment need to be updated for the GLUE/MNLI fine tuning task you perform in this example.

You do this by creating a YAML configuration file that will be referenced when invoking a training script. As noted, when describing [the common training driver](https://github.com/tensorflow/models/blob/master/official/nlp/docs/train.md) you will be able to fine tune these settings even further for each training run by using the `--params_override` flag.

The configuration file has three sections: `task`, `trainer`, `runtime`.

The `task` section contains settings specific to your machine learning task, including a URI to a pre-trained BERT model,  training and validation datasets settings, and evaluation metrics. The `trainer` section configures the settings that control the custom training loop, like checkpoint interval or a number of training steps. The `runtime` section includes the settings for a training runtime: a distributed training strategy, GPU configurations, etc.

In [18]:
%%writefile training_image/trainer/glue_mnli_matched.yaml

task:
  hub_module_url: 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'
  model:
    num_classes: 3
  init_checkpoint: ''
  metric_type: 'accuracy'
  train_data:
    drop_remainder: true
    global_batch_size: 32
    input_path: ''
    is_training: true
    seq_length: 128
    label_type: 'int'
  validation_data:
    drop_remainder: false
    global_batch_size: 32
    input_path: ''
    is_training: false
    seq_length: 128
    label_type: 'int'
trainer:
  checkpoint_interval: 3000
  optimizer_config:
    learning_rate:
      polynomial:
        # 100% of train_steps.
        decay_steps: 36813
        end_learning_rate: 0.0
        initial_learning_rate: 3.0e-05
        power: 1.0
      type: polynomial
    optimizer:
      type: adamw
    warmup:
      polynomial:
        power: 1
        # ~10% of train_steps.
        warmup_steps: 3681
      type: polynomial
  steps_per_loop: 1000
  summary_interval: 1000
  # Training data size 392,702 examples, 3 epochs.
  train_steps: 36813
  validation_interval: 6135
  # Eval data size = 9815 examples.
  validation_steps: 307
  best_checkpoint_export_subdir: 'best_ckpt'
  best_checkpoint_eval_metric: 'cls_accuracy'
  best_checkpoint_metric_comp: 'higher'
runtime:
  distribution_strategy: 'multi_worker_mirrored'
  all_reduce_alg: 'nccl'

Writing training_image/trainer/glue_mnli_matched.yaml


### Build the container

In the next cells, you build the container and push it to Google Container Registry.

In [19]:
TRAIN_IMAGE = f'gcr.io/{PROJECT_ID}/mnli_finetuning'

In [20]:
! docker build -t {TRAIN_IMAGE} training_image

Sending build context to Docker daemon  9.216kB
Step 1/7 : FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-5
 ---> 0f998c784cd6
Step 2/7 : WORKDIR /
 ---> Using cache
 ---> f5d7eda55d06
Step 3/7 : RUN apt remove -y google-fast-socket &&  echo "deb https://packages.cloud.google.com/apt google-fast-socket main" | tee /etc/apt/sources.list.d/google-fast-socket.list &&  curl -s -L https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - &&  apt update && apt install -y google-reduction-server
 ---> Using cache
 ---> 9597ef44789a
Step 4/7 : RUN pip install tf-models-official==2.5.0 tensorflow-text==2.5.0
 ---> Using cache
 ---> c59ec29f72cf
Step 5/7 : COPY trainer /trainer
 ---> Using cache
 ---> 2049c3adbf16
Step 6/7 : ENTRYPOINT ["python"]
 ---> Using cache
 ---> 9f9222a65fb4
Step 7/7 : CMD ["-c", "print('TF Model Garden')"]
 ---> Using cache
 ---> 9bdcb22df83d
Successfully built 9bdcb22df83d
Successfully tagged gcr.io/curious-entropy-222019/mnli_finetuning:latest


In [21]:
! docker push {TRAIN_IMAGE}

Using default tag: latest
The push refers to repository [gcr.io/curious-entropy-222019/mnli_finetuning]

fcd5a0a1: Preparing 
11bd61a3: Preparing 
af86f7b4: Preparing 
40ebdbd6: Preparing 
54d2bd94: Preparing 
a056d495: Preparing 
cb8c2687: Preparing 
43de6bca: Preparing 
87a4088d: Preparing 
522d97b4: Preparing 
519f0898: Preparing 
6aeeabc0: Preparing 
ebfdebb3: Preparing 
6b863e43: Preparing 
43fb2f7a: Preparing 
a9d68143: Preparing 
84ac5c5d: Preparing 
e1798c0c: Preparing 
9de06c8b: Preparing 
59f353b4: Preparing 
3c11e857: Preparing 
5264beff: Preparing 
14beba01: Preparing 
f85bc8aa: Preparing 
dd6c9734: Preparing 
a732d388: Preparing 
78e3bf48: Preparing 
7a4088d: Waiting g 
87e0621d: Preparing 
7ad6008c: Preparing 
22d97b4: Waiting g 
872b888e: Preparing 
512fd434: Preparing 
9d68143: Waiting g 
8308da3d: Layer already exists latest: digest: sha256:9053ebc7539074a50eddc6151609874c8ef7b0782cb05276399535d58f382ac7 size: 7673


### Create a custom training job

When you run a distributed training job with Vertex AI, you specify multiple machines (nodes) in a training cluster. The training service allocates the resources for the machine types you specify. Your running job on a given node is called a replica. A group of replicas with the same configuration is called a worker pool. Vertex Training provides 4 worker pools to cover the different types of machine tasks. To use the Reduction Server, you'll need to use 3 of the 4 available worker pools.

* **Worker pool 0** configures the Primary, chief, scheduler, or "master".  This worker generally takes on some extra work such as saving checkpoints and writing summary files. There is only ever one chief worker in a cluster, so your worker count for worker pool 0 will always be 1.

* **Worker pool 1** is where you configure the rest of the workers for your cluster. 
 
* **Worker pool 2** manages Reduction Server reducers. 

Worker pools 0 and 1 run the custom training container you created in the previous step. Worker pool 2 uses the Reduction Server image provided by Vertex AI.

The helper function below creates a custom job specification using the described worker pool topology.

In [22]:
def prepare_custom_job_spec(
    job_name,
    image_uri,
    args,
    cmd, 
    replica_count=1,
    machine_type='n1-standard-4',
    accelerator_count=0,
    accelerator_type='ACCELERATOR_TYPE_UNSPECIFIED',
    reduction_server_count=0,
    reduction_server_machine_type='n1-highcpu-16',
    reduction_server_image_uri='us-docker.pkg.dev/vertex-ai-restricted/training/reductionserver:latest'
):

    if accelerator_count > 0:
        machine_spec = {
            'machine_type': machine_type,
            'accelerator_type': accelerator_type,
            'accelerator_count': accelerator_count,
        }
    else:
        machine_spec = {
            'machine_type': machine_type
        }
    
    container_spec = {
        'image_uri': image_uri,
        'args': args,
        'command': cmd,
    }
    
    chief_spec = {
        'replica_count': 1,
        'machine_spec': machine_spec,
        'container_spec': container_spec
    }

    worker_pool_specs = [chief_spec]
    if replica_count > 1:
        workers_spec = {
            'replica_count': replica_count - 1,
            'machine_spec': machine_spec,
            'container_spec': container_spec
        }
        worker_pool_specs.append(workers_spec)
        
    if reduction_server_count > 1:
        workers_spec = {
            'replica_count': reduction_server_count,
            'machine_spec': {
                'machine_type': reduction_server_machine_type,
            },
            'container_spec': {
                'image_uri': reduction_server_image_uri
            }
        }
        worker_pool_specs.append(workers_spec)
        
    custom_job_spec = {
        'display_name': job_name,
        'job_spec': {
            'worker_pool_specs': worker_pool_specs
        }
    }
    
    return custom_job_spec

#### Configure worker pools

When choosing the number and type of reducers, you should consider the network bandwidth supported by a reducer replica’s machine type. In GCP, a VM’s machine type defines its maximum possible egress bandwidth. For example, the egress bandwidth of the `n1-highcpu-16` machine type is limited at 32 Gbps.

Because reducers perform a very limited function, aggregating blocks of gradients, they can run on relatively low-powered and cost effective machines. Even with a large number of gradients this computation does not require accelerated hardware or high CPU or memory resources. However, to avoid network bottlenecks, the total aggregate bandwidth of all replicas in the reducer worker pool must be greater or equal to the total aggregate bandwidth of all replicas in worker pools 0 and 1, which host the GPU workers.

Refer to [the article](TBD) for more information about configuring worker pools when using Reduction Server.

In [30]:
REPLICA_COUNT = 8
WORKER_MACHINE_TYPE = 'a2-highgpu-8g'
#WORKER_MACHINE_TYPE = 'n1-standard-8'
ACCELERATOR_TYPE = 'NVIDIA_TESLA_A100'
#ACCELERATOR_TYPE = 'NVIDIA_TESLA_V100'
PER_MACHINE_ACCELERATOR_COUNT = 8
PER_REPLICA_BATCH_SIZE = 32
#PER_REPLICA_BATCH_SIZE = 16

REDUCTION_SERVER_COUNT = 0
REDUCTION_SERVER_MACHINE_TYPE = 'n1-highcpu-16'

#### Fine tune the  GLUE/MNLI experiment settings

The default settings for the GLUE/MNLI fine tuning experiment have been configured in the YAML configuration file created in the previous steps. To override the defaults for a specific training run you can use the `--params_override` flag.

The `params_override`  accepts a string with comma separated key/value pairs for each parameter to be overridden.

In the following cell you update the following settings:

- `trainer.train_step` - The number of training steps. 
- `trainer.steps_per_loop` - The training script prints out updates about training progress every `steps_per_loop`
- `trainer.summary_interval` - The training script logs Tensorboard summaries every `summary_interval`
- `trainer.validation_interval` - The training script runs validation every `validation_interval`
- `trainer.checkpoint_interval` - The training script creates a checkpoint every `checkpoint_interval`
- `task.train_data.global_batch_size` - Global batch size for training data.
- `task.validation_data.global_batch_size` - Global batch size for validation data
- `task.train_data.input_path` - Location of the training dataset
- `task.validation_data.input_path` - Location of the validation dataset
- `runtime.num_gpus` -Number of GPUs to use on each worker.

In [31]:
PARAMS_OVERRIDE = ','.join([
    'trainer.train_steps=2000',
    'trainer.steps_per_loop=100',
    'trainer.summary_interval=100',
    'trainer.validation_interval=2000',
    'trainer.checkpoint_interval=2000',
    'task.train_data.global_batch_size=' + str(REPLICA_COUNT*PER_REPLICA_BATCH_SIZE),
    'task.validation_data.global_batch_size=' + str(REPLICA_COUNT*PER_REPLICA_BATCH_SIZE*PER_MACHINE_ACCELERATOR_COUNT), 
    'task.train_data.input_path=' + TRAIN_FILE,
    'task.validation_data.input_path=' + EVAL_FILE,
    'runtime.num_gpus=' + str(PER_MACHINE_ACCELERATOR_COUNT),
])

#### Create custom job spec

After the experimentation and configuration parameters have been defined, you create the custom job spec.

In [32]:
JOB_NAME = 'MNLI_{}'.format(time.strftime('%Y%m%d_%H%M%S'))
MODEL_DIR = f'{BUCKET_NAME}/{JOB_NAME}/model'

WORKER_CMD = ['python', 'trainer/train.py']
WORKER_ARGS = [
    '--experiment=bert/sentence_prediction',
    '--mode=train',
    '--model_dir=' + MODEL_DIR,
    '--config_file=trainer/glue_mnli_matched.yaml',
    '--params_override=' + PARAMS_OVERRIDE,
]

custom_job_spec = prepare_custom_job_spec(
    job_name=JOB_NAME,
    image_uri=TRAIN_IMAGE,
    args=WORKER_ARGS,
    cmd=WORKER_CMD,
    replica_count=REPLICA_COUNT,
    machine_type=WORKER_MACHINE_TYPE,
    accelerator_count=PER_MACHINE_ACCELERATOR_COUNT,
    accelerator_type=ACCELERATOR_TYPE,
    reduction_server_count=REDUCTION_SERVER_COUNT,
    reduction_server_machine_type=REDUCTION_SERVER_MACHINE_TYPE,
)

In [33]:
# Examine the spec

pp = pprint.PrettyPrinter()
print(pp.pformat(custom_job_spec))

{'display_name': 'MNLI_20210711_172343',
 'job_spec': {'worker_pool_specs': [{'container_spec': {'args': ['--experiment=bert/sentence_prediction',
                                                                 '--mode=train',
                                                                 '--model_dir=gs://jk-rs-testing/MNLI_20210711_172343/model',
                                                                 '--config_file=trainer/glue_mnli_matched.yaml',
                                                                 '--params_override=trainer.train_steps=2000,trainer.steps_per_loop=100,trainer.summary_interval=100,trainer.validation_interval=2000,trainer.checkpoint_interval=2000,task.train_data.global_batch_size=256,task.validation_data.global_batch_size=256,task.train_data.input_path=gs://jk-rs-testing/datasets/MNLI/mnli_train.tf_record,task.validation_data.input_path=gs://jk-rs-testing/datasets/MNLI/mnli_valid.tf_record,runtime.num_gpus=8'],
                                

### Submit and monitor the job

Use the Vertex AI job client to submit and monitor a training job. 

To submit the job, use the job client service's `create_custom_job` method.

In [34]:
options = dict(api_endpoint=API_ENDPOINT)
client = JobServiceClient(client_options=options)

parent = f"projects/{PROJECT_ID}/locations/{REGION}"

response = client.create_custom_job(
    parent=parent, custom_job=custom_job_spec
)

response

/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


name: "projects/697926852371/locations/us-central1/customJobs/464068673712160768"
display_name: "MNLI_20210711_172343"
job_spec {
  worker_pool_specs {
    machine_spec {
      machine_type: "a2-highgpu-8g"
      accelerator_type: NVIDIA_TESLA_A100
      accelerator_count: 8
    }
    replica_count: 1
    disk_spec {
      boot_disk_type: "pd-ssd"
      boot_disk_size_gb: 100
    }
    container_spec {
      image_uri: "gcr.io/curious-entropy-222019/mnli_finetuning"
      command: "python"
      command: "trainer/train.py"
      args: "--experiment=bert/sentence_prediction"
      args: "--mode=train"
      args: "--model_dir=gs://jk-rs-testing/MNLI_20210711_172343/model"
      args: "--config_file=trainer/glue_mnli_matched.yaml"
      args: "--params_override=trainer.train_steps=2000,trainer.steps_per_loop=100,trainer.summary_interval=100,trainer.validation_interval=2000,trainer.checkpoint_interval=2000,task.train_data.global_batch_size=256,task.validation_data.global_batch_size=256,ta

Use the job client service's `get_custom_job` method to retrieve information about a running job.

In [28]:
client.get_custom_job(name=response.name)

name: "projects/697926852371/locations/us-central1/customJobs/4892233007324200960"
display_name: "MNLI_20210711_164022"
job_spec {
  worker_pool_specs {
    machine_spec {
      machine_type: "a2-highgpu-8g"
      accelerator_type: NVIDIA_TESLA_A100
      accelerator_count: 8
    }
    replica_count: 1
    disk_spec {
      boot_disk_type: "pd-ssd"
      boot_disk_size_gb: 100
    }
    container_spec {
      image_uri: "gcr.io/curious-entropy-222019/mnli_finetuning"
      command: "python"
      command: "trainer/train.py"
      args: "--experiment=bert/sentence_prediction"
      args: "--mode=train"
      args: "--model_dir=gs://jk-rs-testing/MNLI_20210711_164022/model"
      args: "--config_file=trainer/glue_mnli_matched.yaml"
      args: "--params_override=trainer.train_steps=2000,trainer.steps_per_loop=100,trainer.summary_interval=100,trainer.validation_interval=2000,trainer.checkpoint_interval=2000,task.train_data.global_batch_size=32,task.validation_data.global_batch_size=32,tas

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:


In [ ]:
# Delete Cloud Storage objects that were created
! gsutil -m rm -r {BUCKET_NAME}

## Temporary - Upload logs to Tensorboard

In [29]:
print('TENSORBOARD={}'.format('projects/895222332033/locations/us-central1/tensorboards/5983067289333792768'))
print('LOGDIR={}'.format(MODEL_DIR))
print('EXPERIMENT={}'.format(JOB_NAME))
print('./tb-gcp-uploader --tensorboard_resource_name $TENSORBOARD   --logdir=$LOGDIR   --experiment_name=$EXPERIMENT --one_shot=True')

TENSORBOARD=projects/895222332033/locations/us-central1/tensorboards/5983067289333792768
LOGDIR=gs://jk-rs-testing/MNLI_20210711_164022/model
EXPERIMENT=MNLI_20210711_164022
./tb-gcp-uploader --tensorboard_resource_name $TENSORBOARD   --logdir=$LOGDIR   --experiment_name=$EXPERIMENT --one_shot=True
